In [1]:
versioninfo()

Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 12 × Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, skylake)
  Threads: 1 on 12 virtual cores


## 8-5. マクロ

### 8-5-1. Juliaのマクロ呼び出し

#### 仮想コード8-2. マクロ呼び出しの書式

```julia
@somemacro0
@somemacro0()

@somemacro1 《式1》
@somemacro1(《式1》)

@somemacro2 《式1》 《式2》
@somemacro2(《式1》, 《式2》)

# :《以下同様》
```

#### コード8-23. `@__MODULE__` マクロの例

In [2]:
@__MODULE__

Main

In [3]:
@__MODULE__()

Main

In [4]:
getproperty(@__MODULE__, :Int)  # OK

Int64

In [5]:
getproperty(@__MODULE__(), :Int)  # これもOK

Int64

In [6]:
@__MODULE__ === Main  # これはNG

LoadError: syntax: "===" is not a unary operator

In [7]:
@__MODULE__() === Main  # これならOK

true

In [8]:
(@__MODULE__) === Main  # これもOK

true

In [9]:
Main === @__MODULE__  # これもOK

true

#### コード8-24. `@time`/`@timed` マクロの例

In [10]:
function fib(n)
    if n ≤ 1
        n
    else
        fib(n - 2) + fib(n - 1)
    end
end

fib (generic function with 1 method)

In [11]:
@time fib(40)  # `@time(fib(40))` でも同様

  0.388956 seconds


102334155

In [12]:
stats = @timed fib(40)

(value = 102334155, time = 0.392079089, bytes = 0, gctime = 0.0, gcstats = Base.GC_Diff(0, 0, 0, 0, 0, 0, 0, 0, 0))

In [13]:
stats.value

102334155

In [14]:
@timed(fib(40)).time  # `(@timed fib(40)).time`  でも同様

0.389732584

#### コード8-25. `@show` マクロの例

In [15]:
a, b = 1, 2;

In [16]:
@show a

a = 1


1

In [17]:
@show a + b  # == `@show(a + b)`

a + b = 3


3

In [18]:
@show a b  # == `@show(a, b)`

a = 1
b = 2


2

In [19]:
@show a, b  # == `@show((a, b))`

(a, b) = (1, 2)


(1, 2)

### 8-5-2. マクロの定義

#### コード8-26. `@time_ns` マクロの定義例（コード1-6.再掲）

In [20]:
macro time_ns(ex)
    ex = esc(ex)
    quote
        t0 = time_ns()
        val = $ex
        t1 = time_ns()
        println("elapsed time: ", Int(t1-t0), " nanoseconds")
        val
    end
end

@time_ns (macro with 1 method)

#### コード8-27. マクロ `@qte` の定義

In [21]:
macro qte(ex)
    QuoteNode(ex)
end

@qte (macro with 1 method)

#### コード8-28. マクロ `@qte` の使用例(1)

In [22]:
@qte fib(40)

:(fib(40))

In [23]:
@qte a + b

:(a + b)

In [24]:
@qte(:a)

:(:a)

In [25]:
@qte begin
    x = 1
    y = 2
    x + y
end

quote
    #= In[25]:2 =#
    x = 1
    #= In[25]:3 =#
    y = 2
    #= In[25]:4 =#
    x + y
end

#### コード8-29. マクロ `@qte` の使用例(2)

In [26]:
@qte $(1 + 2)

:($(Expr(:$, :(1 + 2))))

In [27]:
qx = :x; qy = :y;
ex5_add_q = @qte $qx + $qy

:($(Expr(:$, :qx)) + $(Expr(:$, :qy)))

In [28]:
args = (:x, :y, 5);
ex5_muladd_q = @qte(muladd($(args...)))

:(muladd($(Expr(:$, :(args...)))))

#### コード8-30. マクロ `@qte` の定義その2（多重定義）

In [29]:
macro qte(ex1, ex2...)
    QuoteNode((ex1, ex2...))
end

@qte (macro with 2 methods)

#### コード8-31. マクロ `@qte` の使用例(3)

In [30]:
methods(var"@qte")

# 2 methods for macro "@qte":
[1] var"@qte"(__source__::LineNumberNode, __module__::Module, ex) in Main at In[21]:1
[2] var"@qte"(__source__::LineNumberNode, __module__::Module, ex1, ex2...) in Main at In[29]:1

In [31]:
@qte a + b  # 1つの式（演算式）⇒引数1つのメソッドが呼ばれる

:(a + b)

In [32]:
@qte a + b c - d  # 2つの式⇒引数2つ以上のメソッドが呼ばれる

(:(a + b), :(c - d))

In [33]:
@qte(1, :a, sin(π))  # 3つの引数が指定されたものと見なされ、引数2つ以上のメソッドが呼ばれる

(1, :(:a), :(sin(π)))

In [34]:
@qte 1, :a, sin(π)  # 「カンマ区切りで式を並べた式（つまりタプル）」と見なされ、引数1つのメソッドが呼ばれる

:((1, :a, sin(π)))

### 8-5-3. マクロの動作原理

#### コード8-32. `@macroexpand` マクロの紹介

In [35]:
@macroexpand @__MODULE__

Main

In [36]:
@macroexpand @qte fib(40)

:($(QuoteNode(:(fib(40)))))

In [37]:
@macroexpand @. A * sin(x) + b

:((+).((*).(A, sin.(x)), b))

In [38]:
@macroexpand @eval $qx + $qy

:(Core.eval(Main, Core._expr(:call, :+, qx, qy)))

#### コード8-33. `@rand1to10` マクロの定義

In [39]:
macro rand1to10()
    rand(1:10)
end

@rand1to10 (macro with 1 method)

#### コード8-34. `@rand1to10` マクロの動作確認

In [40]:
@macroexpand @rand1to10

8

In [41]:
@macroexpand @rand1to10

2

In [42]:
@macroexpand @rand1to10

8

In [43]:
@rand1to10

4

In [44]:
@rand1to10

10

In [45]:
@rand1to10

6

In [46]:
sample_rand1to10(DUMMY...) = @rand1to10

sample_rand1to10 (generic function with 1 method)

In [47]:
sample_rand1to10()

7

In [48]:
sample_rand1to10()

7

In [49]:
sample_rand1to10()

7

In [50]:
sample_rand1to10(1)

7

In [51]:
sample_rand1to10(1, 2, 3)

7

In [52]:
sample_rand1to10(Symbol)

7

#### コード8-35. `@assert` マクロの例

In [53]:
@macroexpand @assert iseven(a)

:(if iseven(a)
      nothing
  else
      Base.throw(Base.AssertionError("iseven(a)"))
  end)

In [54]:
function a_div_2(a)
    @assert iseven(a)
    a ÷ 2
end

a_div_2 (generic function with 1 method)

In [55]:
a_div_2(100)

50

In [56]:
a_div_2(101)

LoadError: AssertionError: iseven(a)

#### 仮想コード8-3. `@assert` マクロを含む関数と同等の関数定義

```julia
function a_div_2(a)
    if iseven(a)
        nothing
    else
        throw(AssertionError("iseven(a)"))
    end
    a ÷ 2
end
```

#### コード8-36. マクロを含む関数定義への `@macroexpand` の適用

In [57]:
@macroexpand(function a_div_2(a)
    @assert iseven(a)
    a ÷ 2
end) |> Base.remove_linenums!

:(function a_div_2(a)
      if iseven(a)
          nothing
      else
          Base.throw(Base.AssertionError("iseven(a)"))
      end
      a ÷ 2
  end)

### 8-5-4. 衛生的なマクロ

#### コード8-26. `@time_ns` マクロの定義例（再掲）

In [58]:
macro time_ns(ex)
    ex = esc(ex)
    quote
        t0 = time_ns()
        val = $ex
        t1 = time_ns()
        println("elapsed time: ", Int(t1-t0), " nanoseconds")
        val
    end
end

@time_ns (macro with 1 method)

#### コード8-37. `@time_ns` マクロの実行例(1)

In [59]:
@time_ns 1 + 1

elapsed time: 7729 nanoseconds


2

In [60]:
@time_ns time_ns()

elapsed time: 5300 nanoseconds


0x00273451868930ad

#### コード8-38. `@time_ns` マクロの実行例(2)

In [61]:
function chk_t0()
    @time_ns t0 += 1000000
end

chk_t0 (generic function with 1 method)

In [62]:
chk_t0()

LoadError: UndefVarError: t0 not defined

In [63]:
@macroexpand(function chk_t0()
    @time_ns t0 += 1000000
end) |> Base.remove_linenums!

:(function chk_t0()
      begin
          var"#59#t0" = Main.time_ns()
          var"#60#val" = (t0 += 1000000)
          var"#61#t1" = Main.time_ns()
          Main.println("elapsed time: ", Main.Int(var"#61#t1" - var"#59#t0"), " nanoseconds")
          var"#60#val"
      end
  end)

#### コード8-39. `gensym()`/`@gensym` の実行例

In [64]:
gensym()

Symbol("##312")

In [65]:
gensym("ABC")  # `gensym(:ABC)` でもOK

Symbol("##ABC#313")

In [66]:
gensym("x", "y", "z")  # こちらは各引数シンボルは受け付けないので注意！

(Symbol("##x#314"), Symbol("##y#315"), Symbol("##z#316"))

In [67]:
@gensym x y z  # この呼び出し自体は値がない（`nothing` が返る）

In [68]:
(x, y, z)

(Symbol("##x#317"), Symbol("##y#318"), Symbol("##z#319"))

#### コード8-40. `gensym()`（`@gensym`）利用例

In [69]:
macro use_gensym()
    @gensym result
    quote
        $result = rand()
    end
end

@use_gensym (macro with 1 method)

In [70]:
@macroexpand @use_gensym

quote
    #= In[69]:4 =#
    var"#62###result#320" = Main.rand()
end

### 8-5-5. 実例

#### コード8-41. `OSTypes.jl`（`OSTypes` モジュールの定義）

In [71]:
module OSTypes

export OSType, Linux, Mac, Windows, OtherOS, @genostype

abstract type OSType end

OSType(osname) = OSType(Val(Symbol(osname)))

struct OtherOS <: OSType
    name::Symbol
end
OSType(::Val{other}) where {other} = OtherOS(other)
Base.print(io::IO, other::OtherOS) = print(io, other.name)

function _genostype(typename)
    osname = string(typename)
    typeid = esc(typename)
    basetype = esc(:(OSTypes.OSType))
    quote
        struct $typeid <: $basetype end
        $basetype(::Val{$(QuoteNode(typename))}) = $typeid()
        Base.print(io::IO, ::$typeid) = print(io, $osname)
    end
end

macro genostype(typename::Symbol)
    _genostype(typename)
end

macro genostype(typenames::Symbol...)
    Expr(:block, (_genostype(typename) for typename in typenames)...)
end

@genostype Linux Mac Windows

end

Main.OSTypes

#### コード8-42. `OSTypes` モジュールの使用例（`@genostype` の動作例）

In [72]:
using .OSTypes

In [73]:
Linux()

Linux()

In [74]:
[OSType(osname) for osname in (:Linux, :Mac, :Windows, :FreeBSD, :Haiku)]

5-element Vector{OSType}:
 Linux()
 Mac()
 Windows()
 OtherOS(:FreeBSD)
 OtherOS(:Haiku)

In [75]:
@genostype FreeBSD  # 現在のモジュールに新しい型 `FreeBSD` が追加される

In [76]:
[OSType(osname) for osname in (:Linux, :Mac, :Windows, :FreeBSD, :Haiku)]

5-element Vector{OSType}:
 Linux()
 Mac()
 Windows()
 FreeBSD()
 OtherOS(:Haiku)

In [77]:
OSType.((:Linux, :Mac, :Windows, :FreeBSD, :Haiku)) .|> string

("Linux", "Mac", "Windows", "FreeBSD", "Haiku")

### 8-5-6. 非標準文字列リテラルとコマンドリテラル

#### 仮想コード8-4. `r"～"`/`raw"～"` の定義例

```julia
# `r"～"` の定義（例）
macro r_str(pattern, flags...)
    Regex(pattern, flags...)
end

# `raw"～"` の定義（例）
macro raw_str(str)
    str
end
```

#### コード8-43. `XX"～"` の確認（`@XX "～"` との比較）

In [78]:
dump(:(XX"abc"))

Expr
  head: Symbol macrocall
  args: Array{Any}((3,))
    1: Symbol @XX_str
    2: LineNumberNode
      line: Int64 1
      file: Symbol In[78]
    3: String "abc"


In [79]:
dump(:(@XX_str "abc"))

Expr
  head: Symbol macrocall
  args: Array{Any}((3,))
    1: Symbol @XX_str
    2: LineNumberNode
      line: Int64 1
      file: Symbol In[79]
    3: String "abc"


#### コード8-44. オリジナル非標準文字列リテラル `basedint"～"xx` の例

In [80]:
macro basedint_str(str, base=10)
    parse(Int, str, base=base)
end

@basedint_str (macro with 2 methods)

In [81]:
basedint"12"  # 10進表記の `12`

12

In [82]:
basedint"12"3  # 3進表記の `12`

5

In [83]:
basedint"12"16  # 16進表記の `12`

18

In [84]:
basedint"12"25  # 25進表記の `12`

27

#### コード8-45. 正規表現オブジェクト（コンストラクタ呼び出し）と正規表現リテラルのパフォーマンス比較

In [85]:
using Random

In [86]:
lines = [randstring(['0':'9';'A':'Z'], 100) for _ in 1:10000];

In [87]:
@time for line in lines
    m = match(Regex("\\d{4}"), line)
    m === nothing || print(devnull, m.match)
end

  0.059904 seconds (50.75 k allocations: 1.843 MiB, 6.15% compilation time)


In [88]:
@time for line in lines
    m = match(r"\d{4}", line)
    m === nothing || print(devnull, m.match)
end

  0.007568 seconds (36.73 k allocations: 1.345 MiB)
